## **ktrain** is a lightweight wrapper library for TensorFlow Keras. It can be very helpful in building projects consisting of neural networks. Using this wrapper, we can build, train and deploy deep learning and machine learning models.

In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 981 kB 52.5 MB/s 
     |████████████████████████████████| 263 kB 53.9 MB/s 
     |████████████████████████████████| 2.8 MB 11.1 MB/s 
     |████████████████████████████████| 1.2 MB 59.3 MB/s 
     |████████████████████████████████| 468 kB 66.0 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 3.3 MB 60.9 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 880 kB 62.6 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.0-py3-none-any.whl size=25312260 sha256=e8b6aa7a1afc793a672fef29e89cd6f941f77aebd08497e7ee50c0756c0f7f26
  Stored in directory: /root/.cache/pip/wheels/da/51/15/f25a223a50ac222ac2bde2c9d52a2a6c5352c46ac7e4625b85
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=9875d0c7e

In [3]:
categories = ['alt.atheism', 'soc.religion.christian',
             'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
train_b = fetch_20newsgroups(subset='train',
   categories=categories, shuffle=True, random_state=42)
test_b = fetch_20newsgroups(subset='test',
   categories=categories, shuffle=True, random_state=42)

In [4]:
print('size of training set: %s' % (len(train_b['data'])))
print('size of validation set: %s' % (len(test_b['data'])))
print('classes: %s' % (train_b.target_names))

size of training set: 2257
size of validation set: 1502
classes: ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


In [5]:
x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

In [6]:
import ktrain
from ktrain import text
#we are going to use the text from the ktrain

STEP 1: Load and Preprocess the Data
Preprocess the data using the texts_from_array function (since the data resides in an array). If your documents are stored in folders or a CSV file you can use the texts_from_folder or texts_from_csv functions, respectively.

In [7]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=train_b.target_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


STEP 2: Load the BERT Model

In [8]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


STEP 3: Train the Model
We train using one of the three learning rates recommended in the BERT paper: 5e-5, 3e-5, or 2e-5. Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking learner.lr_find() and learner.lr_plot(), prior to training. The learner.fit_onecycle method employs a 1cycle learning rate policy.

In [9]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
377/377 [==============================] - 221s 530ms/step - loss: 0.5986 - accuracy: 0.7829
Epoch 2/4
377/377 [==============================] - 208s 551ms/step - loss: 0.1057 - accuracy: 0.9668
Epoch 3/4
377/377 [==============================] - 208s 551ms/step - loss: 0.0345 - accuracy: 0.9920
Epoch 4/4
377/377 [==============================] - 208s 551ms/step - loss: 0.0042 - accuracy: 0.9996


We can use the learner.validate method to test our model against the validation set. As we can see

In [10]:
learner.validate(val_data=(x_test, y_test), class_names=train_b.target_names)

                        precision    recall  f1-score   support

           alt.atheism       0.94      0.90      0.92       319
         comp.graphics       0.97      0.98      0.98       389
               sci.med       0.98      0.94      0.96       396
soc.religion.christian       0.93      0.99      0.96       398

              accuracy                           0.96      1502
             macro avg       0.96      0.95      0.95      1502
          weighted avg       0.96      0.96      0.96      1502



array([[288,   4,   5,  22],
       [  5, 381,   2,   1],
       [ 11,   6, 373,   6],
       [  3,   1,   1, 393]])

https://analyticsindiamag.com/a-complete-guide-to-ktrain-a-wrapper-for-tensorflow-keras/


How to Use Our Trained BERT Model
earner.get_predictor method to obtain a Predictor object capable of making predictions on new raw data.
     

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [12]:
predictor.get_classes()

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [13]:
predictor.predict(test_b.data[0:1])

['sci.med']

In [14]:
print(test_b.data[0])

From: brian@ucsd.edu (Brian Kantor)
Subject: Re: HELP for Kidney Stones ..............
Organization: The Avant-Garde of the Now, Ltd.
Lines: 12
NNTP-Posting-Host: ucsd.edu

As I recall from my bout with kidney stones, there isn't any
medication that can do anything about them except relieve the pain.

Either they pass, or they have to be broken up with sound, or they have
to be extracted surgically.

When I was in, the X-ray tech happened to mention that she'd had kidney
stones and children, and the childbirth hurt less.

Demerol worked, although I nearly got arrested on my way home when I barfed
all over the police car parked just outside the ER.
	- Brian



In [15]:
print(test_b.target_names[test_b.target[0]])

sci.med


predictor.save and ktrain.load_predictor methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.